In [ ]:
pip install transformers datasets torchaudio soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets transformers noisereduce librosa evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00


In [ ]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 49.5 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.5 MB/s eta 0:00:00


In [ ]:
!pip install noisereduce

In [ ]:
# Extract the dataset
!tar -zxvf /content/drive/MyDrive/cv-corpus-12.0-delta-2022-12-07-en.tar.gz -C /content/

Streaming output truncated to the last 5000 lines.
cv-corpus-12.0-delta-2022-12-07/en/clips/common_voice_en_36452452.mp3
cv-corpus-12.0-delta-2022-12-07/en/clips/common_voice_en_36453434.mp3
cv-corpus-12.0-delta-2022-12-07/en/clips/common_voice_en_36453435.mp3
cv-corpus-12.0-delta-2022-12-07/en/clips/common_voice_en_36453436.mp3
cv-corpus-12.0-delta-2022-12-07/en/clips/common_voice_en_36453437.mp3
cv-corpus-12.0-delta-2022-12-07/en/clips/common_voice_en_36453438.mp3
cv-corpus-12.0-delta-2022-12-07/en/clips/common_voice_en_36453449.mp3
cv-corpus-12.0-delta-2022-12-07/en/clips/common_voice_en_36453450.mp3
cv-corpus-12.0-delta-2022-12-07/en/clips/common_voice_en_36453451.mp3
cv-corpus-12.0-delta-2022-12-07/en/clips/common_voice_en_36453452.mp3
cv-corpus-12.0-delta-2022-12-07/en/clips/common_voice_en_36453453.mp3
cv-corpus-12.0-delta-2022-12-07/en/clips/common_voice_en_36453454.mp3
cv-corpus-12.0-delta-2022-12-07/en/clips/common_voice_en_36453455.mp3
cv-corpus-12.0-delta-2022-12-07/en/clip

In [ ]:
import os
import pandas as pd

# Load the metadata
metadata_path = '/content/cv-corpus-12.0-delta-2022-12-07/en/validated.tsv'
clips_path = '/content/cv-corpus-12.0-delta-2022-12-07/en/clips/'
metadata = pd.read_csv(metadata_path, sep='\t')

# Filter the dataset to include only audio files listed in validated.tsv
metadata = metadata[['path', 'sentence']]
metadata['path'] = metadata['path'].apply(lambda x: os.path.join(clips_path, x))

# Reduce the dataset to 3000 samples (2500 train + 500 test)
metadata = metadata.sample(n=3000, random_state=42).reset_index(drop=True)


In [ ]:
import torchaudio
import noisereduce as nr
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

def preprocess_audio(file_path):
    audio, sample_rate = torchaudio.load(file_path)
    audio = audio.squeeze().numpy()

    # Reduce noise
    audio = nr.reduce_noise(y=audio, sr=sample_rate)

    # Normalize audio
    audio = (audio - audio.mean()) / audio.std()

    return audio, sample_rate

# Preprocess each audio file and save the processed audio back to metadata
processed_audios = []
for idx, row in metadata.iterrows():
    audio, sample_rate = preprocess_audio(row['path'])
    input_values = processor(audio, sampling_rate=16000).input_values[0]
    labels = processor.tokenizer(row['sentence']).input_ids
    processed_audios.append({
        "input_values": input_values,
        "input_length": len(input_values),
        "labels": labels
    })

processed_metadata = pd.DataFrame(processed_audios)


In [ ]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt"
        )

        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt"
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [ ]:
from sklearn.model_selection import train_test_split

train_metadata, temp_metadata = train_test_split(processed_metadata, test_size=0.2)
val_metadata, test_metadata = train_test_split(temp_metadata, test_size=0.5)


In [ ]:
from torch.utils.data import Dataset

class CommonVoiceDataset(Dataset):
    def __init__(self, metadata):
        self.metadata = metadata

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        item = self.metadata.iloc[idx]
        return {
            "input_values": item["input_values"],
            "labels": item["labels"]
        }

train_dataset = CommonVoiceDataset(train_metadata)
val_dataset = CommonVoiceDataset(val_metadata)
test_dataset = CommonVoiceDataset(test_metadata)


In [ ]:
import numpy as np
from datasets import load_metric

wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    pred_str = [s.lower() for s in pred_str]

    label_ids = pred.label_ids
    # replace padding with -100
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, group_tokens=False)
    label_str = [s.lower() for s in label_str]

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}


<ipython-input-15-ceec1a5958ce>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


The repository for wer contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wer.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


The repository for cer contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/cer.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
from transformers import Wav2Vec2ForCTC, TrainingArguments, Trainer

model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h",
                                       ctc_loss_reduction="mean",
                                       pad_token_id=processor.tokenizer.pad_token_id)

training_args = TrainingArguments(
  output_dir="./wav2vec2-graduationnn",
  group_by_length=True,
  per_device_train_batch_size=4,  # Reduced batch size to prevent OOM
  evaluation_strategy="steps",
  max_steps=2000,  # Set the maximum number of steps
  save_steps=500,  # Save every 100 steps
  eval_steps=500,  # Evaluate every 500 steps
  logging_steps=500,  # Log every 500 steps
  learning_rate=1e-4,
  warmup_steps=500,
  save_total_limit=2,
  logging_dir='./logs',
  report_to="tensorboard",  # enable logging to TensorBoard
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics
)


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

In [ ]:
trainer.train()
model_save_path = "./wav2vec2-fine-tuned-common-voice"
trainer.save_model(model_save_path)
processor.save_pretrained(model_save_path)


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Step,Training Loss,Validation Loss,Wer,Cer
500,3.436800,0.870985,1.000000,1.000000
1000,1.229100,1.005060,1.000000,1.000000
1500,0.875700,0.891625,1.000000,1.000000
2000,0.874900,0.840459,1.000000,1.000000


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using 

[]

In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Load pre-trained model and processor
model = Wav2Vec2ForCTC.from_pretrained("/content/drive/MyDrive/wav2vec2-common-voice-laitho")
processor = Wav2Vec2Processor.from_pretrained("/content/drive/MyDrive/wav2vec2-common-voice-laitho")

# Load audio
audio_input, sample_rate = torchaudio.load("/content/youtube_6669ad4d1fcae.wav")
if sample_rate != 16000:
    audio_input = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(audio_input)

# Ensure the input tensor is the correct shape
input_values = processor(audio_input.squeeze().numpy(), return_tensors="pt", padding="longest").input_values

# Perform inference
with torch.no_grad():
    logits = model(input_values).logits

# Decode the logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
print(transcription)


OSError: Incorrect path_or_model_id: '/content/drive/MyDrive/wav2vec2-common-voice-laitho'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
print(f"Processor Feature Extractor: {processor.feature_extractor}")
print(f"Processor Tokenizer: {processor.tokenizer}")






In [ ]:
results = trainer.evaluate()
print(results)


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


{'eval_loss': 0.8385899662971497, 'eval_wer': 1.0, 'eval_cer': 1.0, 'eval_runtime': 21.6612, 'eval_samples_per_second': 13.85, 'eval_steps_per_second': 1.754, 'epoch': 1.6666666666666665}
